In [2]:
!pip install peft
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install trl
!pip install langchain
!pip install langchain_community
!pip install rouge-score
!pip install py7zr
!pip install evaluate
!pip install detoxify

Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: docstring-parser
    Found existing installation: docstring-parser 0.15
    Uninstalling docstring-parser-0.15:
      Successfully uninstalled docstring-parser-0.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 20.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 9.7 MB/s e

In [3]:
# Imports
import torch
from detoxify import Detoxify
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric, concatenate_datasets, DatasetDict
from sklearn.model_selection import train_test_split
from peft import get_peft_model, PeftModel, PeftConfig, LoraConfig, TaskType
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from tqdm import tqdm
from rouge_score import rouge_scorer
import numpy as np
from random import randrange
from evaluate import load
from trl.core import LengthSampler
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2024-06-28 05:04:36.739632: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 05:04:36.739737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-28 05:04:36.849694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
dataset = load_dataset("samsum")

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
peft_model = PeftModel.from_pretrained(base_model, 
                                       "/kaggle/input/models/lora_model", 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=True)

ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,                                                               
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True)

reward_model = Detoxify('original')

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

...

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt
100%|██████████| 418M/418M [00:01<00:00, 302MB/s] 
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
class SamsumDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=512):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        dialogue = item['dialogue']
        summary = item['summary']
        
        inputs = self.tokenizer("summarize: " + dialogue, 
                                truncation=True, 
                                max_length=self.max_length, 
                                return_tensors="pt")
        
        return {
            "input_ids": inputs.input_ids.squeeze(),
            "attention_mask": inputs.attention_mask.squeeze(),
            "query": dialogue,
            "summary": summary
        }

train_dataset = SamsumDataset(dataset['train'], tokenizer)

In [6]:
def collator(data):
    return {key: [d[key] for d in data] for key in data[0]}

# PPO Configuration
config = PPOConfig(
    model_name=model_name,
    learning_rate=1e-5,
    cliprange=0.1,
    mini_batch_size=4,
    batch_size=4,
    adap_kl_ctrl=True,  # Enable adaptive KL control
    init_kl_coef=0.2,  # Initial KL coefficient
    target_kl=0.01  # Target KL divergence
)

ppo_trainer = PPOTrainer(
    config=config, 
    model=ppo_model, 
    tokenizer=tokenizer, 
    dataset=train_dataset, 
    data_collator=collator
)

generation_kwargs = {
    "min_length": 5,
    "max_new_tokens": 100,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

In [11]:
steps_per_epoch = len(train_dataset) // config.batch_size
num_epochs = 1
max_ppo_steps = steps_per_epoch * num_epochs

progress_bar = tqdm(total=max_ppo_steps, desc="Training Progress", position=0, leave=True)

for step, batch in enumerate(ppo_trainer.dataloader):
    if step >= max_ppo_steps:
        break   

    prompt_tensors = batch["input_ids"]

    # Get response from PEFT LLM
    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)
        summary_tensors.append(summary.squeeze())
        
    batch["response"] = [tokenizer.decode(r, skip_special_tokens=True) for r in summary_tensors]

    # Compute reward outputs
    results = reward_model.predict(batch["response"])
    rewards = [1 - results['toxicity'][i] for i in range(len(batch["response"]))]
    reward_tensors = [torch.tensor(reward) for reward in rewards]

    # Run PPO step
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
        
    ppo_trainer.log_stats(stats, batch, reward_tensors)
    
    progress_bar.set_postfix({
            'kl': f"{stats['objective/kl']:.3f}",
            'mean': f"{stats['ppo/returns/mean']:.3f}",
            'adv_mean': f"{stats['ppo/policy/advantages_mean']:.3f}"
        })
    
    if step%100 == 0:
        print(f'kl: {stats["objective/kl"]}')
        print(f'mean: {stats["ppo/returns/mean"]}')
        print(f'advantages_mean: {stats["ppo/policy/advantages_mean"]}')
        print('-' * 100)
        
    # Update the progress bar
    progress_bar.update(1)
    
progress_bar.close()

Training Progress:   0%|          | 1/3683 [00:06<7:03:29,  6.90s/it, kl=4.218, mean=0.199, adv_mean=0.000]

kl: 4.218080520629883
mean: 0.19915342330932617
advantages_mean: 2.0604074890684387e-08
----------------------------------------------------------------------------------------------------


Training Progress:   0%|          | 3/3683 [00:17<5:51:10,  5.73s/it, kl=0.253, mean=0.379, adv_mean=-0.000]/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -1.44 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Training Progress:   1%|          | 20/3683 [01:40<5:27:04,  5.36s/it, kl=8.966, mean=-0.284, adv_mean=0.000] /opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -1.68 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Training Progr

kl: -0.20313525199890137
mean: 0.41730809211730957
advantages_mean: 0.0
----------------------------------------------------------------------------------------------------


Training Progress:   3%|▎         | 112/3683 [09:01<5:14:07,  5.28s/it, kl=4.662, mean=0.047, adv_mean=0.000]  /opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -1.16 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Training Progress:   3%|▎         | 114/3683 [09:10<4:51:52,  4.91s/it, kl=3.002, mean=0.337, adv_mean=-0.000] /opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -1.60 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Training P

...

Training Progress:  93%|█████████▎| 3413/3683 [4:21:27<20:37,  4.58s/it, kl=-1.171, mean=0.847, adv_mean=0.000]/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -1.54 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Training Progress:  95%|█████████▌| 3501/3683 [4:28:00<15:01,  4.95s/it, kl=2.137, mean=0.689, adv_mean=-0.000] 

kl: 2.1366782188415527
mean: 0.6892489194869995
advantages_mean: -9.691811797551964e-10
----------------------------------------------------------------------------------------------------


Training Progress:  95%|█████████▌| 3515/3683 [4:29:05<12:18,  4.40s/it, kl=0.638, mean=0.805, adv_mean=0.000] /opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -1.49 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Training Progress:  96%|█████████▌| 3522/3683 [4:29:36<11:07,  4.15s/it, kl=3.523, mean=0.447, adv_mean=-0.000]/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -2.36 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Training P

kl: 0.22613763809204102
mean: 0.9105694890022278
advantages_mean: 5.3551048040390015e-08
----------------------------------------------------------------------------------------------------


Training Progress: 100%|██████████| 3683/3683 [4:42:03<00:00,  4.59s/it, kl=-0.497, mean=0.796, adv_mean=-0.000]


In [19]:
save_directory = "/kaggle/working/final_model"
ppo_trainer.model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
torch.save(ppo_trainer.optimizer.state_dict(), "/kaggle/working/optimizer.pt")

# RLHF evaluation

In [7]:
peft_model_path = "/kaggle/input/models/final_model"
config = PeftConfig.from_pretrained(peft_model_path)
ppo_model = PeftModel.from_pretrained(base_model, peft_model_path)

In [12]:
test_dataset = SamsumDataset(dataset['test'], tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=4, collate_fn=collator, shuffle=False)

In [13]:
base_model.eval()
peft_model.eval()
ppo_model.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
            

In [14]:
peft_model = peft_model.to(device)
ppo_model = ppo_model.to(device)

In [44]:
max_eval_steps = 10

base_toxicity_scores = []
rlhf_toxicity_scores = []
examples = []

progress_bar = tqdm(total=max_eval_steps, desc="Evaluation Progress", position=0, leave=True)

for step, batch in enumerate(test_dataloader):
    if step >= max_eval_steps:
        break

    input_ids = pad_sequence(batch['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
    attention_mask = pad_sequence(batch['attention_mask'], batch_first=True, padding_value=0).to(device)
    dialogues = batch['query']

    with torch.no_grad():
        # Generate summaries from base(PEFT) model
        base_summaries = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=100,
        min_length=5,
        do_sample=True,
        temperature=1.2
        )

        # Generate summaries from RLHF model
        rlhf_summaries = ppo_model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=100,
            min_length=5,
            do_sample=True,
            top_p=0.9,
            temperature=1.2
        )

    base_summaries_text = [tokenizer.decode(s, skip_special_tokens=True) for s in base_summaries]
    rlhf_summaries_text = [tokenizer.decode(s, skip_special_tokens=True) for s in rlhf_summaries]

    # Compute toxicity scores
    base_toxicities = reward_model.predict(base_summaries_text)['toxicity']
    rlhf_toxicities = reward_model.predict(rlhf_summaries_text)['toxicity']

    base_toxicity_scores.extend(base_toxicities)
    rlhf_toxicity_scores.extend(rlhf_toxicities)

    # Store examples
    for i in range(len(dialogues)):
        examples.append({
            'dialogue': dialogues[i],
            'peft_summary': base_summaries_text[i],
            'rlhf_summary': rlhf_summaries_text[i],
            'peft_toxicity': base_toxicities[i],
            'rlhf_toxicity': rlhf_toxicities[i]
        })

    progress_bar.update(1)
    progress_bar.set_postfix({
        'PEFT Toxicity': f"{sum(base_toxicities) / len(base_toxicities):.3f}",
        'RLHF Toxicity': f"{sum(rlhf_toxicities) / len(rlhf_toxicities):.3f}"
    })

# Close the progress bar
progress_bar.close()

Evaluation Progress: 100%|██████████| 10/10 [00:39<00:00,  3.90s/it, PEFT Toxicity=0.249, RLHF Toxicity=0.112]


### Quantitative evaluation

In [45]:
final_toxicity_peft = sum(base_toxicities) / len(base_toxicities)
final_toxicity_rlhf = sum(rlhf_toxicities) / len(rlhf_toxicities)

improvement = (final_toxicity_rlhf - final_toxicity_peft)/final_toxicity_peft * 100
print("Percentage improvement in model summary toxicity: ", improvement, "%")

Percentage improvement in model summary toxicity:  -55.074720137557065 %


### Qualitative evaluation

In [74]:
import random
for i, example in enumerate(random.sample(examples, 1)):
    print(f"Dialogue: {example['dialogue']}")
    print(f"\nPEFT Model Summary (Toxicity: {example['peft_toxicity']:.4f}):")
    print(example['peft_summary'])
    print(f"\nRLHF Model Summary (Toxicity: {example['rlhf_toxicity']:.4f}):")
    print(example['rlhf_summary'])
    print("-" * 50)

Dialogue: Rita: I'm so bloody tired. Falling asleep at work. :-(
Tina: I know what you mean.
Tina: I keep on nodding off at my keyboard hoping that the boss doesn't notice..
Rita: The time just keeps on dragging on and on and on.... 
Rita: I keep on looking at the clock and there's still 4 hours of this drudgery to go.
Tina: Times like these I really hate my work.
Rita: I'm really not cut out for this level of boredom.
Tina: Neither am I.

PEFT Model Summary (Toxicity: 0.0034):
Rita is tired and she has snoozed. Ginna is snooying at her work. Rita hates her job, but says she is not ideal for hers. 

RLHF Model Summary (Toxicity: 0.0016):
Rita is very bored at work, she spends too much time looking at the clock. Tina thinks she dislikes her job, she's not cut out for it.
--------------------------------------------------
